<a href="https://colab.research.google.com/github/aditi2259/AF-New/blob/main/new_AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import os
import zipfile
import pandas as pd
import numpy as np
from tensorflow.keras.utils import Sequence
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [41]:
# Load data from the CSV file
data = pd.read_csv('/content/REFERENCE-v0.csv', header=None, names=['id', 'value'])

In [42]:
# Map labels to integers, ensuring that only valid labels are considered
label_mapping = {label: idx for idx, label in enumerate(sorted(set(data['value']))) if isinstance(label, (int, float, str))}
data['mapped_value'] = data['value'].map(label_mapping)

# Filter out any rows with invalid labels
data = data.dropna(subset=['mapped_value'])

In [43]:
# Initialize lists for file paths and labels
file_paths = []
labels = []

# Collect file paths and labels
for _, row in data.iterrows():
    mat_file_path = os.path.join(extraction_path, f"{row['id']}.mat")
    print(f"Checking file: {mat_file_path}")  # Debugging output

    if os.path.exists(mat_file_path):
        file_paths.append(mat_file_path)
        labels.append(int(row['mapped_value']))  # Ensure the label is an integer
    else:
        print(f"File not found and will be skipped: {mat_file_path}")


Streaming output truncated to the last 5000 lines.
Checking file: /content/training2017/training2017/A03529.mat
Checking file: /content/training2017/training2017/A03530.mat
Checking file: /content/training2017/training2017/A03531.mat
Checking file: /content/training2017/training2017/A03532.mat
Checking file: /content/training2017/training2017/A03533.mat
Checking file: /content/training2017/training2017/A03534.mat
Checking file: /content/training2017/training2017/A03535.mat
Checking file: /content/training2017/training2017/A03536.mat
Checking file: /content/training2017/training2017/A03537.mat
Checking file: /content/training2017/training2017/A03538.mat
Checking file: /content/training2017/training2017/A03539.mat
Checking file: /content/training2017/training2017/A03540.mat
Checking file: /content/training2017/training2017/A03541.mat
Checking file: /content/training2017/training2017/A03542.mat
Checking file: /content/training2017/training2017/A03543.mat
Checking file: /content/training20

In [44]:
# Set the number of classes
num_classes = len(label_mapping)

In [45]:
# Proceed with your existing code logic for data processing
class Datagenerator(Sequence):
    def __init__(self, file_paths, labels, batch_size, shuffle=True, target_shape=(9000, 1), num_class=4):
        self.file_paths = file_paths
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.target_shape = target_shape
        self.num_class = num_class
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.file_paths) / self.batch_size))

    def __getitem__(self, idx):
        batch_paths = self.file_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = to_categorical(batch_labels, num_classes=self.num_class)
        batch_data = [self.load_mat_file(path) for path in batch_paths]
        return tf.convert_to_tensor(batch_data), np.array(batch_labels)

    def on_epoch_end(self):
        if self.shuffle:
            data = list(zip(self.file_paths, self.labels))
            np.random.shuffle(data)
            self.file_paths, self.labels = zip(*data)

    def load_mat_file(self, path):
        ecg_dict = loadmat(path)
        ecg_data = ecg_dict['val'].flatten()
        ecg_data = np.pad(ecg_data, (0, max(0, self.target_shape[0] - len(ecg_data))), 'constant')
        ecg_data = ecg_data[:self.target_shape[0]]
        ecg_data = ecg_data.reshape(self.target_shape)
        return ecg_data

# Set batch size
batch_size = 32

In [46]:
# Split the dataset into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(
    file_paths,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

# Create data generators
train_generator = Datagenerator(train_paths, train_labels, batch_size=batch_size)
print('Training generator created')
val_generator = Datagenerator(val_paths, val_labels, batch_size=batch_size)
print('Validation generator created')

Training generator created
Validation generator created


In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

def CNN_model(input_shape):
    model = Sequential()
    model.add(tf.keras.Input(shape=input_shape))  # Use Input layer
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))

    model.add(Dense(num_classes, activation='softmax'))  # Update output layer to use num_classes

    return model

In [ ]:
input_shape = (9000, 1)
model = CNN_model(input_shape)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, validation_data=val_generator, epochs=10)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


214/214 ━━━━━━━━━━━━━━━━━━━━ 1193s 6s/step - accuracy: 0.4552 - loss: 8.0236 - val_accuracy: 0.4754 - val_loss: 1.7573
Epoch 2/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 1170s 5s/step - accuracy: 0.5493 - loss: 1.2757 - val_accuracy: 0.5950 - val_loss: 0.9927
Epoch 3/10
 79/214 ━━━━━━━━━━━━━━━━━━━━ 11:29 5s/step - accuracy: 0.5984 - loss: 1.0327